In [58]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import keras
import tensorflow as tf
from keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [59]:
def building_block(filters,block):
    
#     import random
#     bn = random.randint(100,300,)
    #判断block1和2
    if block != 0: #如果不等于0 那么使用 stride=1
        stride = 1
    else:         #如果等于0 采用stride 2 两倍下采样 也就是 如果是 building1 使用2倍下采样
        stride = 2

    def f(x):
        #主通路结构
        y = KL.Conv2D(filters=filters,kernel_size=(1,1),strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)

        y = KL.Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(y) #注意这里没有stride使用padding same就是保证size相同
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)

        #主通路输出
        y = KL.Conv2D(filters=4*filters,kernel_size=(1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)

        #判断是哪个block 设定不同的 shortcut支路参数
        if block == 0 : #如果是0 那么就是block1的通路
            shortcut = KL.Conv2D(filters=4*filters,kernel_size=(1,1),strides=stride)(x)
            shortcut = KL.BatchNormalization(axis=3)(shortcut)
        else:
            #如果不等于0 那就是block2  那么就直接接input的tensor
            shortcut = x

        #主通路和shortcut 相加
        y = KL.Add()([y,shortcut]) #y主 shortcut支路 直接通过add层相加
        import random
        y = KL.Activation('relu',name='last'+str(random.randint(100,300)))(y)
        return y
    return f

In [60]:
#resnet 主输入函数
def ResNet_Extractor(inputs):
    x = KL.Conv2D(filters=64,kernel_size=(3,3),padding='same')(inputs)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation('relu')(x)

    #控制调用网络结构feature map 特征图
    #每个stage要有不同的 b12的数量 ，还有 第一个Block1 输入维度后边要迭代（stage）
    filters = 64
    block = [2,2,2]
    for i,block_num in enumerate(block):
        for block_id in range(block_num):
            x = building_block(filters=filters,block=block_id)(x)
        filters *= 2 #每个stage double filter个数

    return x

In [61]:
#share map 和 anchor提取

In [62]:
def RpnNet(featuremap, k=9):
    #共享
    shareMap = KL.Conv2D(filters=256,kernel_size=(3,3),padding='same',name='SSharemap')(featuremap)
    shareMap = KL.Activation('linear')(shareMap)
    
    #计算rpn分类前后景
    rpn_classification = KL.Conv2D(filters=2*k,kernel_size=(1,1))(shareMap)
    rpn_classification = KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,2]))(rpn_classification)
    rpn_classification = KL.Activation('linear',name='rpn_classification')(rpn_classification)
    
    rpn_probability = KL.Activation('softmax',name='rpn_probability')(rpn_classification)
    
    #计算回归修正
    
    rpn_position = KL.Conv2D(filters=4*k,kernel_size=(1,1))(shareMap)
    rpn_position = KL.Activation('linear')(rpn_position)
    rpn_BoundingBox =KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,4]),name='rpn_POS')(rpn_position)
    
    return rpn_classification,rpn_probability,rpn_BoundingBox
     
    

In [63]:
# x = KL.Input((64,64,3)) 
# featureMap = ResNet_Extractor(x)
# rpn_classification,rpn_probability,rpn_BoundingBox = RpnNet(featureMap,k=9)
# model = Model(inputs = [x],outputs=[rpn_classification,rpn_probability,rpn_BoundingBox])
# model.summary()
# plot_model(model=model,to_file='siezemap test.png',show_shapes=True)

In [64]:
def RPNClassLoss(rpn_match,rpn_Cal):
    rpn_match = tf.squeeze(rpn_match,axis=-1)
    
    indices = tf.where(K.not_equal(x=rpn_match,y=0))
    #1=1  0 and -1 = 0
    anchor_class =  K.cast(K.equal(rpn_match,1),tf.int32) #return Ture = 1 False = 0 1 1010010 
    
    anchor_class = tf.gather_nd(params=anchor_class ,indices=indices) #这个是我们原始样本结果
    
    rpn_cal_class = tf.gather_nd(params=rpn_Cal,indices=indices) # 这个我们rpn计算值结果
     
    loss = K.sparse_categorical_crossentropy(target=anchor_class,output=rpn_cal_class,from_logits=True)
                # if                        then                         else     
    loss = K.switch(condition=tf.size(loss)>0,then_expression=K.mean(loss),else_expression=tf.constant(0.0))
    
    return loss



In [65]:
#小工具提取
def batch_pack(x,counts,num_rows):
    output = []
    for i in range(num_rows):
        output.append(x[i,:counts[i]])
    return tf.concat(output,axis=0)

In [67]:
#位置loss
def RpnBBoxLoss(target_bbox,rpn_match,rpn_bbox):
    rpn_match = tf.squeeze(input=rpn_match,axis=-1)
    indice =  tf.where(K.equal(x = rpn_match,y=1)) #正样本 位置
    
    rpn_bbox = tf.gather_nd(params=rpn_bbox,indices=indice) #rpn 预测值
    
    batch_counts = K.sum(K.cast(K.equal(x = rpn_match,y=1),tf.int32),axis=-1)
    target_bbox = batch_pack(x= target_bbox,counts=batch_counts,num_rows=10)
    
    #loss 计算
    
    diff = K.abs(target_bbox-rpn_bbox)
    less_than_one = K.cast(K.less(x = diff, y=1.0),tf.float32)
    loss = less_than_one * 0.5 * diff**2 + (1 - less_than_one)*(diff-0.5)
    
    loss = K.switch(condition=tf.size(loss)>0,then_expression=K.mean(loss),else_expression=tf.constant(0.0))

    return loss
    
    

In [68]:
#确定input
input_image = KL.Input(shape=[64,64,3],dtype=tf.float32)
input_bbox = KL.Input(shape=[None,4],dtype=tf.float32)
input_class_ids = KL.Input(shape = [None],dtype=tf.int32)    # map {'dog':0,'cat':1}
input_rpn_match = KL.Input(shape=[None,1],dtype=tf.int32)
input_rpn_bbox = KL.Input(shape=[None,4],dtype=tf.float32)

In [69]:
#in out put
feature_map = ResNet_Extractor(input_image)
rpn_classification,rpn_probability,rpn_BoundingBox = RpnNet(feature_map,k=9)

loss_rpn_class = KL.Lambda(lambda x:RPNClassLoss(*x),name='classloss')([input_rpn_match,rpn_classification])
loss_rpn_bbox = KL.Lambda(lambda x:RpnBBoxLoss(*x),name='bboxloss')([input_rpn_bbox,input_rpn_match,rpn_BoundingBox])

model = Model(inputs=[input_image,input_bbox,input_class_ids,input_rpn_match,input_rpn_bbox],
              outputs = [rpn_classification,rpn_probability,rpn_BoundingBox,loss_rpn_class,loss_rpn_bbox] )




In [71]:
#自定义loss 输入
loss_layer1 = model.get_layer('classloss').output
loss_layer2 = model.get_layer('bboxloss').output

model.add_loss(tf.reduce_mean(loss_layer1))
model.add_loss(tf.reduce_mean(loss_layer2))

model.compile(loss=[None]*len(model.outputs),
             optimizer=keras.optimizers.SGD(lr=0.00003))

#编译模型
 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_117 (Conv2D)             (None, 64, 64, 64)   1792        input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_111 (BatchN (None, 64, 64, 64)   256         conv2d_117[0][0]                 
__________________________________________________________________________________________________
activation_72 (Activation)      (None, 64, 64, 64)   0           batch_normalization_111[0][0]    
__________________________________________________________________________________________________
conv2d_118

## Demo 一下 batch-pack

In [54]:
import numpy as np
import tensorflow as tf
def batch_pack(x,counts,num_rows): #matix
    output=[]
    for i in range(num_rows):
        output.append(x[i,:counts[i]]) #i第几行  ， 取该行的范围,count 对应第几行[i] 的预留数量
    return tf.concat(output,axis=0)

In [55]:
x = np.array(
        [[1,2,0,0,0],
         [3,6,0,0,0],
         [4,7,0,0,0],
         [5,8,0,0,0],
         [2,1,0,0,0],
        ]
)

counts = np.array([2,1,1,1,1])
num_rows = 5

In [56]:
picked = batch_pack(x=x,counts=counts,num_rows=num_rows)

In [33]:
with tf.Session() as sess:
    print (sess.run(picked))

[1 2 3 4 5 2]
